In [ ]:
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
import time
from langdetect.detector_factory import DetectorFactory

In [ ]:
factory = DetectorFactory()
factory.load_profile('ld_profile')
 
def detect(text):
    detector = factory.create()
    detector.append(text)
    return detector.detect()
 
def detect_langs(text):
    detector = factory.create()
    detector.append(text)
    return detector.get_probabilities()

factory.get_lang_list()

In [ ]:
f = open("count_courses_coursera.txt",'r') # в данном файле должна быть записана последняя страница, которую мы знаем
oldpg = f.read()
oldpg = eval(oldpg)
f.close()
#Случай, когда страниц столько же, но возможно на странице прибавилось курсов(появились новые)
#Всего на странице может находиться 35 курсов

f = open("coursera_courses.txt",'r')
last_course = f.readlines()[-1]
last_course = eval(last_course)[0]
f.close()
url1 = "https://www.coursera.org/directory/courses?page=" +str(oldpg)
page = requests.get(url1)
soup = BeautifulSoup(page.content, 'html.parser')
st1 = soup.find_all('a', class_ ="MuiTypography-root MuiLink-root MuiLink-underlineHover css-19sqvu6 MuiTypography-colorPrimary")
f = open("coursera_courses.txt",'a')
for j in range(0, len(st1)):
    #course-title
    ct1 = soup.find_all('a', class_ ="MuiTypography-root MuiLink-root MuiLink-underlineHover css-19sqvu6 MuiTypography-colorPrimary")[j].get_text()
    print(ct1[0:ct1.find('(')-1],len(ct1[0:ct1.find('(')-1]))
    print(last_course,len(last_course))
    if ct1[0:ct1.find('(')-1] == last_course:
        print('DONE')
        break;
print('START')
for k in range(j+1,len(st1)):
    ct1 = soup.find_all('a', class_ ="MuiTypography-root MuiLink-root MuiLink-underlineHover css-19sqvu6 MuiTypography-colorPrimary")[k].get_text()
    print(ct1)
    if detect(ct1[0:ct1.find('(')-1]) == 'ru':
        try:
            #course URL
            curl1 = soup.find_all("a", class_='MuiTypography-root MuiLink-root MuiLink-underlineHover css-19sqvu6 MuiTypography-colorPrimary')[k]
            f.write(f"('''{ct1.split('(')[0].strip()}''', 'https://www.coursera.org{curl1.get('href')}', '''{ct1.split('(')[1][:-1].strip()}''')\n")
                     #f.write("https://www.coursera.org"+ curl.get('href') + '\n')
        except:
             continue
f.close()
#берем юрл на каталог, заходим проверяем какая страница последняя 
#допустим последний раз у нас было 221 страниц, сейчас заходим и теперь их 222 
url = "https://www.coursera.org/directory/courses"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
newpg = soup.find_all('a',class_ ="MuiTypography-root MuiLink-root MuiLink-underlineHover box number css-bu128x MuiTypography-colorPrimary")[-1].get_text()

f1 = open("coursera_courses.txt",'a')

#Случай, когда появилась новая страница. Нужно парсить эту страницу отдельно и добавлять ее в файл с остальными
if newpg > oldpg:
    for i in range(oldpg+1,newpg+1):
        url1 = "https://www.coursera.org/directory/courses?page=" +str(i)
        page = requests.get(url1)
        soup = BeautifulSoup(page.content, 'html.parser')
        st1 = soup.find_all('a', class_ ="MuiTypography-root MuiLink-root MuiLink-underlineHover css-19sqvu6 MuiTypography-colorPrimary")
        for j in range(0, len(st1)):
            #course-title
            ct1 = soup.find_all('a', class_ ="MuiTypography-root MuiLink-root MuiLink-underlineHover css-19sqvu6 MuiTypography-colorPrimary")[j].get_text()
            #русский язык чек
            if detect(ct1[0:ct1.find('(')-1]) == 'ru':
                try:
                    #course URL
                    curl1 = soup.find_all("a", class_='MuiTypography-root MuiLink-root MuiLink-underlineHover css-19sqvu6 MuiTypography-colorPrimary')[j]
                    f1.write(f"('''{ct1.split('(')[0].strip()}''', 'https://www.coursera.org{curl1.get('href')}', '''{ct1.split('(')[1][:-1].strip()}''')\n")
                        #f.write("https://www.coursera.org"+ curl.get('href') + '\n')
                except:
                    continue
f1 = open("count_courses_coursera.txt",'w')
f1.write(str(newpg))
f1.close()
f.close()